# LWCF Grant Funding in New England

The following analysis relies on grant information by state for 1965-2011 compiled by [Investigate West](http://www.invw.org/2012/06/11/lwcf-grants-database-1283/). Unfortunately, the Land and Water Conservation Fund (LWCF) [Project List by County and Summary Reports](https://waso-lwcf.ncrc.nps.gov/public/index.cfm) page appears to be down. Additionally, the latest Annual Report posted on nps.gov is from 2012.

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [22]:
# Import data sets
ct_grants = pd.read_csv('./Data/CT_LWCFGrants1965-2011.csv')
me_grants = pd.read_csv('./Data/ME_LWCFGrants1965-2011.csv')
ma_grants = pd.read_csv('./Data/MA_LWCFGrants1965-2011.csv')
nh_grants = pd.read_csv('./Data/NH_LWCFGrants1965-2011.csv')
ri_grants = pd.read_csv('./Data/RI_LWCFGrants1965-2011.csv')
vt_grants = pd.read_csv('./Data/VT_LWCFGrants1965-2011.csv')

new_england = pd.concat([ct_grants, me_grants, ma_grants, nh_grants, ri_grants, vt_grants], ignore_index=True)

In [23]:
new_england.head()

,Grant ID & Element,Grant Name,Sponsor,County,State,Grant Amount,Year Approved,Year Completed,Type
0,1 - XXX,SULLIVAN PROPERTY,DEPT. OF AGRICULTURE & NATURAL RESOU,HARTFORD,CT,"$5,000.00",1965.0,1967.0,Acquisition
1,3 - XXX,ADELMAN FIELD,TOWN OF BOZRAH,MIDDLESEX,CT,"$3,000.00",1965.0,1966.0,Acquisition
2,4 - XXX,STATE OUTDOOR RECREATION PLAN,DEPT. OF AGRICULTURE & NATURAL RESOU,PLANNING,CT,"$22,500.00",1965.0,1966.0,Planning
3,5 - XXX,COUTU PROPERTY,TOWN OF MANSFIELD,TOLLAND,CT,$300.00,1965.0,1967.0,Acquisition
4,2 - XXX,GOODWIN STATE FOREST,DEPT. OF AGRICULTURE & NATURAL RESOU,WINDHAM,CT,"$6,000.00",1965.0,1967.0,Acquisition


In [24]:
new_england.tail()

,Grant ID & Element,Grant Name,Sponsor,County,State,Grant Amount,Year Approved,Year Completed,Type
3263,624 - XXX,Tunbridge Recreation Area Acquisition,Town of Tunbridge,ORANGE,VT,"$11,035.00",2011.0,2011.0,Acquisition
3264,625 - XXX,Fair Haven Recreation Area and Playground,Town of Fair Haven,RUTLAND,VT,"$13,000.00",2011.0,2011.0,Development
3265,627 - XXX,Chester Recreation - Community Pavilion Project,Town of Chester,WINDSOR,VT,"$17,000.00",2011.0,2011.0,Development
3266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
new_england.shape

(3268, 9)

In [27]:
new_england.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3268 entries, 0 to 3267
Data columns (total 9 columns):
Grant ID & Element    3260 non-null object
Grant Name            3260 non-null object
Sponsor               3260 non-null object
County                3260 non-null object
State                 3260 non-null object
Grant Amount          3260 non-null object
Year Approved         3260 non-null float64
Year Completed        3260 non-null float64
Type                  3260 non-null object
dtypes: float64(2), object(7)
memory usage: 229.9+ KB


In [30]:
# Remove rows with all NaN values
new_england.dropna(how='all', inplace=True)